# Products Data Wrangling

1.Importing dataset.

2.Checking columns name and deleting unusefull columns.

3.Checking data consistency

4.Checking for duplicates

5.Merging 

6.Exporting data

# 1.Importing Libraries and Data

In [1]:
#Imorting libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import scipy

In [2]:
path = r'C:\Users\poury\12-2023 Instacart Basket Analysis'

In [3]:
df_customers = pd.read_csv(os.path.join(path, '02 Data', 'original data', 'customers.csv'))

In [4]:
df_ords_prods = pd.read_pickle(os.path.join(path,'02 Data/Prepared Data/orders_products_merged_derived_2.pkl'))

# 2.Checking Columns Name 

In [5]:
df_customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [6]:
df_customers.shape

(206209, 10)

In [7]:
#Changing needed columns name
df_customers.rename(columns={"First Name":"first_name", "Surnam":"last_name", "Gender":"gender", "STATE":"state", "Age":"age"}, inplace=True)

In [8]:
df_customers.head()

,user_id,first_name,last_name,gender,state,age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


# Checking Data Consistency

In [9]:
#Checking data types
df_customers.dtypes

user_id          int64
first_name      object
last_name       object
gender          object
state           object
age              int64
date_joined     object
n_dependants     int64
fam_status      object
income           int64
dtype: object

In [10]:
#Changing data types 
df_customers['date_joined'] = pd.to_datetime(df_customers['date_joined'])
df_customers['gender'] = df_customers['gender'].astype('category')
df_customers['state'] = df_customers['state'].astype('category')
df_customers['fam_status'] = df_customers['fam_status'].astype('category')

In [11]:
#Checking for mixed data type
for col in df_customers.columns.tolist():
    weird = (df_customers[[col]].map(type) != df_customers[[col]].iloc[0].apply(type)).any(axis = 1)
    
    if len (df_customers[weird]) > 0:
        print (col)

first_name


In [12]:
#Fixing the mixed type in first_name column
df_customers['first_name'] = df_customers['first_name'].astype(str)

In [13]:
#Missing values
df_customers.isnull().sum()

user_id         0
first_name      0
last_name       0
gender          0
state           0
age             0
date_joined     0
n_dependants    0
fam_status      0
income          0
dtype: int64

#No missing value

In [14]:
#Duplicates
df_dups = df_customers[df_customers.duplicated()]

In [15]:
df_dups

,user_id,first_name,last_name,gender,state,age,date_joined,n_dependants,fam_status,income


#No duplicate

In [16]:
#Checking states
df_customers['state'].nunique()

51

In [17]:
df_customers['state'].value_counts(dropna=False)

state
Alabama                 4044
District of Columbia    4044
Iowa                    4044
Indiana                 4044
Illinois                4044
Idaho                   4044
Georgia                 4044
Florida                 4044
Hawaii                  4044
Delaware                4044
Connecticut             4044
Colorado                4044
California              4044
Arkansas                4044
Arizona                 4044
Alaska                  4044
South Dakota            4043
Ohio                    4043
Oklahoma                4043
Oregon                  4043
Pennsylvania            4043
Rhode Island            4043
South Carolina          4043
Wisconsin               4043
Tennessee               4043
Texas                   4043
Utah                    4043
Vermont                 4043
Virginia                4043
Washington              4043
West Virginia           4043
North Carolina          4043
North Dakota            4043
Missouri                4043
New York

#All the states including D.C are included with right names

In [18]:
df_customers.describe()

,user_id,age,date_joined,n_dependants,income
count,206209.000000,206209.000000,206209,206209.000000,206209.000000
mean,103105.000000,49.501646,2018-08-17 03:06:30.029532928,1.499823,94632.852548
min,1.000000,18.000000,2017-01-01 00:00:00,0.000000,25903.000000
25%,51553.000000,33.000000,2017-10-23 00:00:00,0.000000,59874.000000
50%,103105.000000,49.000000,2018-08-16 00:00:00,1.000000,93547.000000
75%,154657.000000,66.000000,2019-06-10 00:00:00,3.000000,124244.000000
max,206209.000000,81.000000,2020-04-01 00:00:00,3.000000,593901.000000
std,59527.555167,18.480962,NaN,1.118433,42473.786988


#Observations:
Age column looks ok, we have no unnormal age and outlier here.
Other statistics also look ok.

# Merging datas

In [19]:
df_ords_prods.shape

(32434212, 28)

In [20]:
#Checking for the merge column
df_ords_prods.dtypes

product_id                   int64
product_name                object
aisle_id                     int64
department_id                int64
prices                     float64
order_id                     int64
user_id                      int64
eval_set                    object
order_number                 int64
orders_day_of_week           int64
order_hour_of_day            int64
days_since_prior_order     float64
add_to_cart_order            int64
reordered                    int64
_merge                    category
price_range                 object
price_range_loc             object
Busiest_day                 object
Busiest_days                object
Busiest_priod_of_day        object
max_order                    int64
loyalty_flag                object
mean_spend                 float64
spending_flag               object
mean_DSPO                  float64
median_DSPO                float64
median_order_freq          float64
order_freq_flag             object
dtype: object

In [21]:
df_ords_prods.drop(columns=['_merge'], inplace=True)

In [22]:
df_ords_prods['user_id'] = df_ords_prods['user_id'].astype('int64')

In [23]:
df_ords_prods_customers = df_ords_prods.merge(df_customers, on=['user_id'], indicator=True)

In [24]:
df_ords_prods_customers['_merge'].value_counts(dropna=False)

_merge
both          32434212
left_only            0
right_only           0
Name: count, dtype: int64

In [27]:
df_ords_prods_customers.to_pickle(os.path.join(path, '02 Data', 'prepared data', 'ords_prods_customers.pkl'))